In [1]:
import re
import sys,os
sys.path.append('../../../classifier/models/time_pattern/')
from time_pattern import TimePattern

In [2]:
tp = TimePattern()

Time Zone is set from ENV: Asia/Shanghai


In [40]:
s1 = '我可以先还5万2，但要100天后才能还清'
s2 = '我可以先还52000，但要100天后才能还清'
s3 = '我可以先还52，000.23， 但要23点15分后才能还清'
s4 = '我可以先还五万6千零二十'
s5 = '但要100天后才能还清'

In [41]:
clean = s3
clean = re.sub(r',|，| ','',clean)
# remove time

In [42]:
number = re.findall(r'([一二三四五六七八九零十百千万亿]+|[0-9]+[,]*[0-9]+.[0-9]+)',clean)
number

['52000.23', '23点15']

In [44]:
number = re.findall(r'(?:还)[一二三四五六七八九零十百千万亿兆0-9点.]+',clean)
number

['52000.23', '23点15']

In [48]:
tp.process('明天早上11点')

[{'pattern': '明天早上11点',
  'time': datetime.datetime(2018, 8, 9, 11, 0, tzinfo=<DstTzInfo 'Asia/Shanghai' CST+8:00:00 STD>),
  'gapS': 44349.645987,
  'gapH': 12.319346107500001}]

In [63]:
reg = '今天|明天(?:(?:上午)|(?:早上)|(?:早晨)|(?:凌晨))[1一]点'
re.findall(reg,'明天上午一点')

['明天上午一点']

In [82]:
class CHN2NUM:
    def __init__(self):
        self.CN_NUM = {
    '〇' : 0, '一' : 1, '二' : 2, '三' : 3, '四' : 4, '五' : 5, '六' : 6, '七' : 7, '八' : 8, '九' : 9, '零' : 0,
    '壹' : 1, '贰' : 2, '叁' : 3, '肆' : 4, '伍' : 5, '陆' : 6, '柒' : 7, '捌' : 8, '玖' : 9, '貮' : 2, '两' : 2
}

        for i in range(10):
            self.CN_NUM[str(i)] = i
        self.CN_UNIT = {
                            '十' : 10,
                            '拾' : 10,
                            '百' : 100,
                            '佰' : 100,
                            '千' : 1000,
                            '仟' : 1000,
                            '万' : 10000,
                            '萬' : 10000,
                            '亿' : 100000000,
                            '億' : 100000000,
                            '兆' : 1000000000000,
                        }

    def chinese_to_arabic(self,cn:str):
        unit = 0   # current
        ldig = []  # digest
        for cndig in reversed(cn):
            if cndig in self.CN_UNIT:
                unit = self.CN_UNIT.get(cndig)
                if unit == 10000 or unit == 100000000:
                    ldig.append(unit)
                    unit = 1
            else:
                dig = self.CN_NUM.get(cndig)
                if unit:
                    dig *= unit
                    unit = 0
                ldig.append(dig)
        if unit == 10:
            ldig.append(10)
        val, tmp = 0, 0
        for x in reversed(ldig):
            if x == 10000 or x == 100000000:
                val += tmp * x
                tmp = 0
            else:
                tmp += x
        val += tmp
        return val

In [107]:
tt = CHN2NUM()
tt.chinese_to_arabic('两')

2

In [88]:
common_used_numerals={u'零':0,u'一':1,u'二':2,u'三':3,u'四':4,u'五':5,u'六':6,u'七':7,u'八':8,u'九':9,u'十':10,u'百':100,u'千':1000,u'万':10000,u'亿':100000000,u'元':1.0,u'毛':0.1,u'分':0.01,u'两':2,u'角':0.1,u'块':1.0}
numerals = {u'零':0,u'一':1,u'二':2,u'三':3,u'四':4,u'五':5,u'六':6,u'七':7,u'八':8,u'九':9}
units = {u'十':10,u'百':100,u'千':1000,u'万':10000,u'亿':100000000,u'元':1.0,u'毛':0.1,u'分':0.01,u'两':2,u'角':0.1,u'块':1.0}
units_10 = {u'十':u'元',u'百':u'十',u'千':u'百',u'万':u'千',u'亿':u'千万',u'元':u'角',u'毛':u'分',u'角':'分',u'块':u'角'}

def transform(uchars_cn):
    s=uchars_cn
    if len(s) ==0:
        return 0
    if len(s)>2:
        if s[-1] in numerals:
            if s[-2] in units.keys():
                s = s + units_10[s[-2]]
    return float(cn2digit(s))

def cn2digit(uchars_cn):
    s=uchars_cn
    if not s:
        return 0
    for i in [u'亿',u'万',u'千',u'百',u'十',u'元',u'块',u'毛',u'角',u'分']:
        if i in s:
            ps=s.split(i)
            lp=cn2digit(ps[0])
            if lp==0 and i not in [u'元',u'块',u'毛',u'角',u'分']:
                lp=1
            rp=cn2digit(ps[1])
            return 1.0*lp*common_used_numerals.get(i[-1], 0)+rp*1.0
    return 1.0*common_used_numerals.get(s[-1], 0)


assert( transform(u"两千7") == 2700)
assert( transform(u"两万") == 20000)
assert( transform(u"两千") == 2000)
assert( transform(u"两毛七") == 0.27)
assert( transform(u"两毛七分") == 0.27)
assert( transform(u"两千三百万") == 23000000)
assert( transform(u"两千三百万零两毛七分") == 23000000.27)

AssertionError: 

In [105]:
transform(u"十二")

12.0

In [98]:
chs_arabic_map = {'零':0, '一':1, '二':2, '三':3, '四':4,
        '五':5, '六':6, '七':7, '八':8, '九':9,
        '十':10, '百':100, '千':10 ** 3, '万':10 ** 4,
        '〇':0, '壹':1, '贰':2, '叁':3, '肆':4,
        '伍':5, '陆':6, '柒':7, '捌':8, '玖':9,
        '拾':10, '佰':100, '仟':10 ** 3, '萬':10 ** 4,
        '亿':10 ** 8, '億':10 ** 8, '幺': 1,
        '０':0, '１':1, '２':2, '３':3, '４':4,
        '５':5, '６':6, '７':7, '８':8, '９':9}

def convertChineseDigitsToArabic (chinese_digits, encoding="utf-8"):
#     if isinstance (chinese_digits, str):
#         chinese_digits = chinese_digits.decode(encoding)

    result  = 0
    tmp     = 0
    hnd_mln = 0
    for count in range(len(chinese_digits)):
        curr_char  = chinese_digits[count]
        print(curr_char)
        curr_digit = chs_arabic_map.get(curr_char, None)
        print(curr_digit)
        # meet 「亿」 or 「億」
        if curr_digit == 10 ** 8:
            result  = result + tmp
            result  = result * curr_digit
            # get result before 「亿」 and store it into hnd_mln
            # reset `result`
            hnd_mln = hnd_mln * 10 ** 8 + result
            result  = 0
            tmp     = 0
        # meet 「万」 or 「萬」
        elif curr_digit == 10 ** 4:
            result = result + tmp
            result = result * curr_digit
            tmp    = 0
        # meet 「十」, 「百」, 「千」 or their traditional version
        elif curr_digit >= 10:
            tmp    = 1 if tmp == 0 else tmp
            result = result + curr_digit * tmp
            tmp    = 0
        # meet single digit
        elif curr_digit is not None:
            tmp = tmp * 10 + curr_digit
        else:
            return result
    result = result + tmp
    result = result + hnd_mln
    return result

In [101]:
convertChineseDigitsToArabic('二千七')

二
2
千
1000
七
7


2007

In [100]:
chs_arabic_map.get('两')

In [ ]:
a.encode()